<a href="https://colab.research.google.com/github/manufabregas/Dataset-Asistencia-a-turnos-m-dicos/blob/main/TP%202/Pr%C3%A1ctico_2_An%C3%A1lisis_y_curaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo y alcance
En esta segunda aproximación (que suele ir de la mano con el análisis exploratorio y visualización) se pretende empezar a transformar el dataset para los primeros experimentos de aprendedores en las siguientes materias.

En este laboratorio el primer gran objetivo va a ser lograr un único dataset con columnas nuevas, que condense toda la información importante para el posterior entrenamiento y aprendizaje que permitirá conocer si algunas de las variables incluidas en el mismo tienen influencia sobre la problemática que nos atañe.

## Lista de tareas

1.   Verificar la información brindada por el dataset a partir del práctico n°1. ¿Hay algo que les haya llamado la atención y consideran que puede estar mal tomado el dato, o sesgado? La inclusión del mismo, ¿podría generar problemas a futuro? Tomar una decisión por cada dato o información que consideren no fiable y justificar su respuesta.
2.   Verificar y asegurar que no existan datos duplicados.
3.   Verificar que no existan caracteres "raros" para los datos de las variables de especialidad (msp_name) y nombre de especialistas (doc_full_name).
4.   A partir de los valores faltantes reconocidos en el práctico N° 1 se deberán proponer dos soluciones posibles para trabajar con estos datos. En este punto deberán ejecutar ambas soluciones y fundamentar su aplicación. Además, deberán elegir una para obtener su dataset final, explicando su elección.
5.   Hay algunas variables que contienen datos de texto pero no están normalizados, como por ejemplo "msp_name", donde encontramos que ginecología puede estar escrita como "GINECOLOGÍA", "ginecologia", "ginecologa", entre otras variantes. Es importante aplicar alguna técnica para unificar estos datos. Se recomienda el uso de [regex](https://regexr.com/).
6.   Outliers: ¿Existen valores atípicos en las columnas del dataset? A partir del análisis del práctico n°1 aplicar una solución para manejar solo esos outliers.
7.   A partir de la exploración del práctico n°1, ¿qué variables nuevas se les ocurre que pueden generar y sean de importancia pensando en el problema de investigación planteado?. Se espera que puedan construir al menos 3 variables nuevas a partir de la información contenida en la base de datos (pueden usar las generadas en el práctico 1 pero es importante que se justifique por qué son interesantes de considerar respecto al problema).
8.   Reconocer las relaciones entre las distintas variables con la feature target del problema a resolver en el práctico 3.
9.   Si tuvieran que agregar datos que no son posibles de extraer a partir de la información que brinda el dataset, y que pueden suponer una gran influencia en nuestra variable objetivo, ¿Qué datos utilizarían? ¿Cómo plantearían su obtención? (teórico)
10.  A partir del práctico n°1 pudieron observar algunas clases que se encuentran desbalanceadas en el dataset. Pensando en el práctico siguiente y la variable target para predecir la asistencia de turnos médicos (attendance), ¿Consideran que la misma se encuentra desbalanceada? ¿Creen que esto podría afectar de alguna manera la predicción? En el caso de estar desbalanceada, ¿Tratarían de aplicar alguna técnica de balanceo? De ser positiva la respuesta se les invita a explorar las diversas técnicas de balanceo de datos y elegir la que consideren más adecuada para este caso justificando su respuesta. (sólo dejar en el práctico la técnica elegida)
11.   Para trabajar con datos sensibles y confidenciales es de gran importancia utilizar técnicas de despersonalización o anonimización de datos. Si quisieran proteger los nombres de especialistas y hospitales, ¿qué técnicas se les ocurren? Pueden mencionar varias, pero es de gran importancia que pueden aplicar al menos una sobre el dataset (es solo a fines de poner en práctica lo aprendido en clase, luego ese dataset anonimizado no será utilizado para el práctico 3)
12.   Guardar el dataset resultante con un nuevo nombre (sin la técnica de anonimización aplicada en el punto 11) para trabajarlo más adelante.

# Se evaluarán los siguientes aspectos:
  
1- Que se apliquen los conceptos vistos con los profes en el teórico y en el práctico.

2- Que el entregable no sea solo la notebook. El informe debe tener un mensaje claro y debe presentarse en un formato legible para cualquier tipo de stakeholder. Además, se deberá entregar el dataset trabajado.

3- Capacidad de Análisis y justificación. Proactividad en la profundización de los análisis.

4- Criterio para elegir que solución aplicar en cada caso y con qué método implementarla.

5- Calidad y estilo de código.


Deadline tentativo para la entrega 29/07/2024

### **Cargando el dataset**

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as msno
%matplotlib inline

In [49]:
df=pd.read_csv("https://raw.githubusercontent.com/kunan-sa/mentoria_2024/master/dataset/base_asistencia_anon.csv", encoding='utf8')
df=df.drop(columns=['Unnamed: 0'])
df_original = df.copy()
df_original.head(3)

,doc_id,doc_full_name,msp_name,msp_id,hos_id,heq_id,hin_name,hin_id,age_avg,dni_asistance_rate,...,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,action_A_count,action_B_count,action_C_count,app_days_gap,patient_id,event_id,attendance
0,215,Carmen Sánchez,Neurologa,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,77.0,72.727273,...,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0
1,1855,Sofía García,CARDIOLOGIA INFANTIL,141,Instituto Harrenhal,12,OSDE,0,7.0,50.000000,...,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0
2,215,Carmen Sánchez,Neurologa,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,60.0,37.500000,...,71.085359,69.875045,2023-10-10 13:40:00.000,10,0,2,33,72497,320006,0


## **A ESTA VARIABLE VAMOS A REALIZARLE TODAS LAS MODIFICACIONES PERTINENTES DEL DATASET**

In [50]:
df_base=df_original.copy()

#Agregaremos la columna periodo
df_base['periodo'] = pd.to_datetime(df_base['app_start_dt'])
df_base['periodo'] = df_base['periodo'].dt.to_period('M').astype(str)

#Eliminamos msp_id y hin_name	(id propio de cada clínica, va a diferir con cada clínica este nro)
df_base=df_base.drop(['msp_id','hin_name'],axis=1)

#Renombramos columnas
df_base=df_base.rename(columns={'action_A_count':'qt_sac_pac_ac',"action_B_count":"qt_canc_pac_ac",'action_C_count':'qt_cons_pac'})


df_base.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


# **Verificar la información brindada por el dataset a partir del práctico n°1. ¿Hay algo que les haya llamado la atención y consideran que puede estar mal tomado el dato, o sesgado? La inclusión del mismo, ¿podría generar problemas a futuro? Tomar una decisión por cada dato o información que consideren no fiable y justificar su respuesta.**

**RESPUESTA:**
Nos llamó la atención los turnos:

* PONIENTE: Baja tasa de asistencia tanto a nivel hospital, como médicos como especialidades
* Hospitales con especialidades con dos tasas de asistencia
* Medicos con dos identificaciones e incluso por sede




In [51]:
df_uno=df_base.copy()
df_uno.head(2)


,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


# **CONSULTORIO PONIENTE**

---



In [52]:
df_uno_poniente=df_uno.copy()
df_uno_poniente=df_uno_poniente[df_uno_poniente['hos_id']=='Consultorios Poniente']
df_uno_poniente.head(3)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
14,995,Jan Levinson,TRAUMATOLOGIA,Consultorios Poniente,1,0,24.0,50.0,22.519084,22.323462,20.803002,2023-07-24 08:40:00.000,2,0,0,32,22818,416643,0,2023-07
17,939,Joe Dugan,CLINICA MEDICA,Consultorios Poniente,1,0,50.0,40.0,17.877095,17.879418,20.803002,2023-09-14 08:00:00.000,3,0,0,20,6012,428204,0,2023-09
24,945,Daenerys Targaryen,TRAUMATOLOGIA,Consultorios Poniente,1,0,17.0,0.0,22.033898,22.323462,20.803002,2023-10-06 09:40:00.000,4,0,6,37,38304,428800,0,2023-10


In [53]:
import plotly.graph_objects as go
attendance_counts = df_uno_poniente.groupby(['periodo', 'attendance']).size().unstack(fill_value=0)
attendance_counts.columns = ['Inasistencia', 'Asistencia']

fig = go.Figure(data=[
    go.Bar(name='Asistencia', x=attendance_counts.index, y=attendance_counts['Asistencia'], marker_color='green',text=attendance_counts['Asistencia']),
    go.Bar(name='Inasistencia', x=attendance_counts.index, y=attendance_counts['Inasistencia'], marker_color='red',text=attendance_counts['Inasistencia'],)
])
fig.update_layout(barmode='stack', title='CONSULTORIO PONIENTE: Asistencia e Inasistencia por Mes', xaxis_title='Periodo', yaxis_title='Cantidad')

fig.show()


In [71]:
import plotly.graph_objects as go
attendance_counts = df_uno_poniente[['doc_full_name','doc_asistance_rate']].drop_duplicates()

fig = go.Figure(data=[go.Bar(name='Tasa de Asistencia', x=attendance_counts['doc_full_name'], y=attendance_counts['doc_asistance_rate'],
                             marker_color='blue', text=attendance_counts['doc_asistance_rate'].round(2), textposition='auto')])

fig.update_layout(title='Tasa de Asistencia por Médico', xaxis_title='Nombre del Médico', yaxis_title='Tasa de Asistencia (%)', xaxis={'categoryorder': 'total descending'})

fig.show()

In [89]:
import plotly.graph_objects as go
attendance_counts = df_uno_poniente[['msp_name','msp_asistance_rate']].drop_duplicates().sort_values('msp_asistance_rate',ascending=False)

fig = go.Figure(data=[go.Bar(name='Tasa de Asistencia', x=attendance_counts['msp_name'], y=attendance_counts['msp_asistance_rate'],
                             marker_color='lightgray', text=attendance_counts['msp_asistance_rate'].round(2), textposition='auto')])

fig.update_layout(title='Tasa de Asistencia por Especialidad', xaxis_title='Nombre del Especialidad', yaxis_title='Tasa de Asistencia (%)', xaxis={'categoryorder': 'total descending'})

fig.show()

In [87]:
df_check=df_uno_poniente.copy()
df_check=df_check[df_check['msp_name'].isin(['ECOCARDIOGRAMA'])]#,'ORL','OTORRINOLARINGOLOGIA'
df_check

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
5018,1055,Martha M. Masters,ECOCARDIOGRAMA,Consultorios Poniente,1,0,54.0,0.0,34.589041,NaN,20.803002,2023-06-29 10:30:00.000,1,0,0,27,12392,413455,0,2023-06
46575,1055,Martha M. Masters,ECOCARDIOGRAMA,Consultorios Poniente,1,0,34.0,0.0,34.589041,NaN,20.803002,2023-06-08 10:30:00.000,1,0,0,16,16355,412146,0,2023-06


# **HOSPITALES - ESPECIALIDADES**

---



In [98]:
df_asist_dup=df_uno.copy().groupby(['hos_id','msp_name'])['msp_asistance_rate'].nunique()
multiple_asist = df_asist_dup[df_asist_dup > 1]
multiple_asist



hos_id               msp_name      
Instituto Harrenhal  ALERGIA           2
                     CARDIOLOGIA       2
                     CLINICA MEDICA    2
                     NEUMONOLOGIA      2
                     REUMATOLOGIA      2
Name: msp_asistance_rate, dtype: int64

# **MEDICOS -ID**

---



In [104]:
# Controlamos que todos los médicos (612) tengan un solo ID (sin importar hospital y sede)
unique_docs = df_uno.copy().groupby('doc_full_name')['doc_id'].nunique()
multiple_ids = unique_docs[unique_docs > 1]

if multiple_ids.empty:
    print("Todos los médicos tienen un único ID.")
else:
    print("Los siguientes médicos tienen más de un ID:")
    print(multiple_ids)

Los siguientes médicos tienen más de un ID:
doc_full_name
Agustín González     2
Facundo Fernández    2
Franco González      2
Isabella González    2
Lautaro Martínez     2
Martina Pérez        2
Mateo Martínez       2
Matías Fernández     2
Sam Owens            2
Santiago López       2
Sofía García         3
Ted Wheeler          2
Thiago Pérez         2
Valentina López      2
Name: doc_id, dtype: int64


In [103]:
unique_ids_per_group = df_uno.copy().groupby(['doc_full_name', 'hos_id', 'heq_id'])['doc_id'].nunique() # Agrupa doc_full_name, hos_id y heq_id, y cuenta los doc_id únicos

if (unique_ids_per_group == 1).all():
    print("Todos los médicos tienen un único ID por hospital y sede.")
else:
    problematic_medics = unique_ids_per_group[unique_ids_per_group != 1]
    print("Los siguientes médicos tienen más de un ID por hospital y sede:\n")
    print(problematic_medics)

Los siguientes médicos tienen más de un ID por hospital y sede:

doc_full_name     hos_id             heq_id
Lautaro Martínez  Centro Winterfell  2         2
Name: doc_id, dtype: int64


# **Verificar y asegurar que no existan datos duplicados.**

In [90]:
df_dos=df_base.copy()
df_dos.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


In [92]:
duplicados = df_dos.duplicated()

df_duplicados = df_dos[duplicados]
df_duplicados

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo


A simple vista no se ven duplicaciones en los registros.

# **Verificar que no existan caracteres "raros" para los datos de las variables de especialidad (msp_name) y nombre de especialistas (doc_full_name).**

In [105]:
df_tres=df_base.copy()
df_tres.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


In [107]:
df_tres_espec=df_tres.copy()[['hos_id','msp_name']]
df_tres_espec = df_tres_espec.groupby('hos_id')['msp_name'].unique().reset_index()
df_tres_espec

,hos_id,msp_name
0,Centro Winterfell,"[CARDIOLOGIA, GINECOLOGIA Y OBSTETRICIA, TRAUM..."
1,Centro médico Casterly Rock,"[GINECOLOGIA Y OBSTETRICIA, MEDICINA CLINICA, ..."
2,Consultorios Poniente,"[TRAUMATOLOGIA, CLINICA MEDICA, RADIOLOGIA, CA..."
3,Dorne,"[GASTROENTEROLOGIA, TRAUMATOLOGIA INFANTIL, CA..."
4,Hospital Dragonstone,"[Neurologa, Ginecologa, Espirometras, Traumato..."
5,Instituto Harrenhal,"[CARDIOLOGIA INFANTIL, PEDIATRIA, NEUMONOLOGIA..."


Acá partir de la tabla anterior, se pueden observar caracteres en minúscula para el hospital Dragonstone y palabras como 'Neurologa' 'Ginecologa' 'Traumatologa' que en otros hospitales figuran como 'Neurología', 'Ginecología' y 'Traumatología'